In [16]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
# from utilities3 import LpLoss
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import math
from sklearn.model_selection import train_test_split
from functools import reduce
from functools import partial
import operator
from timeit import default_timer
from matplotlib.ticker import FormatStrFormatter
import deepxde as dde
from scipy import linalg
from scipy.interpolate import interp1d

In [17]:
def solveBetaFunction(x, gamma, amp):
    beta = np.zeros(len(x))
    for idx, val in enumerate(x):
        beta[idx] = amp*math.cos(gamma*math.acos(val))
    return beta

def buildF(x, gamma1, amp1, gamma2, amp2):
    b1 = solveBetaFunction(x, gamma1, amp1)
    b2 = solveBetaFunction(x, gamma2, amp2)
    nx = len(x)
    f = np.zeros((nx, nx))
    for idx, val in enumerate(x):
        for idx2, val2 in enumerate(x):
            if idx <= idx2:
                f[idx][idx2] = b1[idx]*b2[idx2]
    return f

def buildtau(nx,tau):
    tau_matrix = tau * np.ones((nx, nx))
    # tau_matrix = np.triu(tau_matrix)
    tau_matrix = tau_matrix.reshape(1, -1)
    return tau_matrix

def buildCof_int_x_1(x, dx):
    nx = len(x)
    cof_int_x_1 = np.triu(np.ones((nx,nx)))
    cof_int_x_1[:,-1]=1/2*np.ones((1,nx))
    cof_int_x_1= dx*(cof_int_x_1-1/2*np.eye(nx))
    return cof_int_x_1

def buildCof_int_0_x(x, dx):
    nx = len(x)
    cof_int_0_x = np.tril(np.ones((nx,nx)))
    cof_int_0_x[:,0]=1/2*np.ones((1,nx))
    cof_int_0_x= dx*(cof_int_0_x-1/2*np.eye(nx))
    return cof_int_0_x

def findinterpolation(kappabud, spatial1, spatial2):
    fx = interp1d(spatial1, kappabud, kind="cubic")
    kappabudInterp = fx(spatial2)
    return kappabudInterp

def zeroToNan(x):
    for j in range(len(x)):
        for i in range(len(x)):
            if i > j:
                x[i][j] = float('nan')
    return x

def solveControl(u, Kbud, Lbud, Jbud, nx, dx, tau_obs, k):
    return (sum(Kbud*u[0:nx]*dx) +tau_obs*sum(Lbud*u[nx:2*nx]*dx)+k*sum(Jbud*u[2*nx:3*nx]*dx)
            - Kbud[0]*u[0]*dx/2
            - Kbud[nx-1]*u[nx-1]*dx/2
            - tau_obs*Lbud[0]*u[nx]*dx/2
            - tau_obs*Lbud[nx-1]*u[2*nx-1]*dx/2
            - k*Jbud[0]*u[2*nx]*dx/2
            - k*Jbud[nx-1]*u[3*nx-1]*dx/2)

def solveOpenLoop(_, _a, _b, _c, _d, _e, _f, _g):
    return 0

def fastKernelCalc(f, c, tau, tau_obs, x, cof_int_x_1, cof_int_0_x):
    nx = len(x) 
    dx=x[2]-x[1]
    k=tau-tau_obs
    Ntau=int(tau/dx)+1
    K = np.zeros((nx, nx))
    result = 0
    for i in range(nx-2, -1, -1):
        result += (f[[i+1],[i+1]] + f[[i],[i]])*dx/2*(-1)
        K[i,i] = result
    for i in range(nx-2, -1, -1):
        num = nx - i
        A1 = np.diag([-2]*num) + np.diag([1]*(num-1),1)
        A1[[0],[0]] = 1
        A1[[0],[1]] = 0
        A1[[-1],[-1]] = 1

        A121=np.zeros((num,num))
        A122=np.zeros((num,num))
        A121[1:num-1,0:num-1]= f.T[i+1:i-1+num,i:i-1+num]
        A122[0:num-1,0:num-1] = dx*cof_int_0_x[0:num-1,0:num-1] 
        A1 = A1+ A121*A122
        B1=np.zeros((num,1))
        B1[[0],[0]] = K[[i],[i]]
        B1[1:num-1,0] =-K[[i+1],i+1:nx-1].reshape((num-2)) +(dx*f[i,i+1:nx-1]).reshape((num-2))
        if i+ Ntau<nx:
            B1[num-1,0]=sum(cof_int_x_1[i+Ntau-1,:]*K[i+Ntau-1,:]*c)-c[i+Ntau-1]
        else:
            B1[num-1,0]=0
        D=np.linalg.solve(A1,B1)
        K[[i],i:nx]=D.T

    J=np.zeros((nx, nx))
    c_matrix=np.repeat(c[np.newaxis,:], nx, 0)
    J[0:nx,0]=np.sum(cof_int_x_1*c_matrix*K, axis=1)-c[0:nx]
    AJ = np.diag([k+1]*nx) + np.diag([-1]*(nx-1),-1)
    AJ[0,0]=1
    for i in range(nx-2, -1, -1):
        BJ=np.zeros((nx,1))
        BJ[0,0]=J[i,0]
        BJ[1:nx,0]=k*J[i+1,1:nx]
        J[[i],0:nx]=(np.linalg.solve(AJ,BJ)).T
    
    L=np.zeros((nx, nx))
    L[0:nx,0]=J[0:nx,-1]
    AL = np.diag([tau_obs+1]*nx) + np.diag([-1]*(nx-1),-1)
    AL[0,0]=1
    for i in range(nx-2, -1, -1):
        BL=np.zeros((nx,1))
        BL[0,0]=L[i,0]
        BL[1:nx,0]=tau_obs*L[i+1,1:nx]
        L[[i],0:nx]=(np.linalg.solve(AL,BL)).T
    return K, L, J

def fastObserverGainCalc(f, tau_obs, a, x, cof_int_x_1):
    nx = len(x) 
    dx = x[2] - x[1]
    F1 = np.zeros((nx, nx))
    result = 0
    for i in range(1, nx, 1):
        result += (f[[i-1],[i-1]] + f[[i],[i]])*dx/2*(-1/a)
        F1[i,i] = result
    for j in range(2, nx, 1):
        num = j
        CC1 = np.diag([2]*num) + np.diag([-1]*(num-1),-1)
        CC1[[-1],[num-2]] = 0
        CC1[[-1],[num-1]] = 1
        CC21=np.zeros((num,num))
        CC22=np.zeros((num,num))
        CC21= f[1:1+num,1:1+num]
        CC22 = -(dx/a)*cof_int_x_1[nx-j:nx,nx-j:nx]
        CC = CC1+ CC21*CC22 
        DD=np.zeros((num,1))
        DD[-1, 0] = F1[j,j]
        DD[0:num-1,0] = F1[1:j,j-1].reshape((num-1)) -(dx/a*f[1:j,j]).reshape((num-1))
        D=np.linalg.solve(CC,DD)
        D2=np.dot(np.linalg.inv(CC),DD)
        F1[1:j+1,j]=D.T

    F23 = np.zeros((nx, nx))
    F23[:,-1]=a*tau_obs*F1[:,-1]
    # EE1 = np.diag([1+a*tau_obs]*(nx-1)) + np.diag([-a*tau_obs]*(nx-2),-1)
    # EE21 = -dx*tau_obs*cof_int_x_1[1:nx,1:nx]
    # EE22 = f[1:nx,1:nx]
    EE = np.diag([1+a*tau_obs]*(nx-1)) + np.diag([-a*tau_obs]*(nx-2),-1)\
        -(dx*tau_obs*cof_int_x_1[1:nx,1:nx]) * f[1:nx,1:nx]
    EE_inv = np.linalg.inv(EE)
    for j in range(nx-2,-1,-1):
        F23[1:nx,j] =  np.dot(EE_inv, F23[1:nx,j+1])
    
    Q1 = -F23[:,0]/tau_obs
    Q2 = -np.flipud(F23[-1,:])
    return F1, F23, Q1, Q2

def solvePDE(tau, tau_obs, c, f, Kbud, Lbud, Jbud, init_condition, x, t, printFreq=500):
    k=tau-tau_obs
    nx = len(x)
    nt = len(t)
    dx=x[1]-x[0]
    dt=t[1]-t[0]
    uu = np.zeros((nt, 3*nx))
    uu[0,:] = init_condition
    for i in range(1, nt):
        if i%int(1000) == 0:
            print("Completed:", i, "/", nt, flush=True)
            
        uu[i][0] = solveControl(uu[i-1], Kbud, Lbud, Jbud, nx, dx,tau_obs,k)
        fres = np.zeros(nx)
        for j in range(1, nx):
            fres[j] = sum(f[j][j:nx]*uu[i-1][j:nx])*dx-f[j][j]*uu[i-1][j]*dx/2-f[j][nx-1]*uu[i-1][nx-1]*dx/2
        uu[i][2*nx-1]=uu[i-1][nx-1]
        uu[i][nx:2*nx-1] = uu[i-1][nx:2*nx-1]+dt/(tau_obs*dx)*(uu[i-1][nx+1:2*nx]-uu[i-1][nx:2*nx-1])
        uu[i][3*nx-1]=uu[i-1][nx]
        uu[i][2*nx:3*nx-1] = uu[i-1][2*nx:3*nx-1]+dt/(k*dx)*(uu[i-1][2*nx+1:3*nx]-uu[i-1][2*nx:3*nx-1])
        uu[i][1:nx] = uu[i-1][1:nx] - dt/dx*(uu[i-1][1:nx] - uu[i-1][0:nx-1]) + dt*fres[1:nx] + dt*c[1:nx]*uu[i-1][nx]
    return uu

def solvePDEwithObse(tau, tau_obs,  c, f, Kbud, Lbud, Jbud,  Q1, Q2, init_condition, init_condition_obs, x, t, printFreq):
    ## tau is state delay
    ## tau_obs is sensor delay
    k=tau-tau_obs
    nx = len(x) 
    dx = x[2] - x[1]
    nt = len(t)
    dt = t[1]-t[0] 
    ###### initialization
    u = np.zeros((nt, 3*nx))
    u_est = np.zeros((nt, 3*nx)) 
    u[0,:] = init_condition
    u_est[0,:] = init_condition_obs
    print("Solving PDE... Timesteps Needed:", nt, flush=True) 
    for i in range(1,nt,1):
        if i%int(printFreq) == 0:
            print("Completed:", i, "/", nt, flush=True)
        u[i,0] = sum(Kbud*u_est[i-1,0:nx]*dx)   \
            - Kbud[0]*u_est[i-1,0]*dx/2 - Kbud[nx-1]*u_est[i-1,nx-1]*dx/2 \
            + sum(tau_obs*Lbud*u_est[i-1,nx:2*nx]*dx )  \
            - tau_obs*Lbud[0]*u_est[i-1,nx]*dx/2 \
            - tau_obs*Lbud[nx-1]*u_est[i-1,2*nx-1]*dx/2 \
            + sum(k*Jbud*u_est[i-1,2*nx:3*nx]*dx ) \
            - k*Jbud[0]*u_est[i-1,2*nx]*dx/2 \
            - k*Jbud[nx-1]*u_est[i-1,3*nx-1]*dx/2  
        fres = np.zeros(nx)
        for j in range(1, nx):
            fres[j] = sum(f[j,j:nx]*u[i-1,j:nx])*dx-f[j][j]*u[i-1][j]*dx/2-f[j][nx-1]*u[i-1][nx-1]*dx/2
        
        u[i][2*nx-1]=u[i-1][nx-1]
        u[i][nx:2*nx-1] = u[i-1][nx:2*nx-1]+dt/(tau_obs*dx)*(u[i-1][nx+1:2*nx]-u[i-1][nx:2*nx-1])
        u[i][3*nx-1]=u[i-1][nx]
        u[i][2*nx:3*nx-1] = u[i-1][2*nx:3*nx-1]+dt/(k*dx)*(u[i-1][2*nx+1:3*nx]-u[i-1][2*nx:3*nx-1])
        u[i][1:nx] = u[i-1][1:nx] - dt/dx*(u[i-1][1:nx] - u[i-1][0:nx-1]) + dt*fres[1:nx] + dt*c[1:nx]*u[i-1][2*nx]
        
        ##### observer part for \hat x
        u_est[i,0] = u[i,0]
        fresu_est = np.zeros(nx)
        for j in range(1, nx):
            fresu_est[j] = sum(f[j,j:nx]*u_est[i-1,j:nx])*dx-f[j][j]*u_est[i-1][j]*dx/2-f[j][nx-1]*u_est[i-1][nx-1]*dx/2
        
        u_est[i,1:nx] = u_est[i-1,1:nx] - dt/dx*(u_est[i-1,1:nx] - u_est[i-1,0:nx-1]) \
            + dt*fresu_est[1:nx] + dt*c[1:nx]*u_est[i-1,2*nx] + dt*Q1[1:nx]*(u[i-1,nx] -u_est[i-1,nx])
        ##### observer part for \hat u1
        u_est[i, 2*nx-1] = u_est[i,nx-1]
        u_est[i,nx:2*nx-1] = u_est[i-1,nx:2*nx-1] + dt/(tau_obs*dx)*(u_est[i-1,nx+1:2*nx]- u_est[i-1,nx:2*nx-1]) \
        + dt*Q2[0:nx-1]/tau_obs * (u[i-1,nx] -u_est[i-1,nx])
        ##### observer part for \hat u2
        u_est[i, 3*nx-1] = u[i,nx]
        u_est[i,2*nx:3*nx-1] = u_est[i-1,2*nx:3*nx-1] + dt/(k*dx)*(u_est[i-1,2*nx+1:3*nx]- u_est[i-1,2*nx:3*nx-1]) 
    return u, u_est

In [18]:
## time
T = 15
dt = 0.001
nt = int(round(T/dt))+1
temporal = np.linspace(0, T, nt)
## space
X = 1
# dx = dt*a
dx = 0.02
nx = int(round(X/dx))+1
spatial = np.linspace(0, X, nx)

cof_int_x_1 = buildCof_int_x_1(spatial, dx)
cof_int_0_x = buildCof_int_0_x(spatial, dx)

## system parameters
# tau = 1.8
# tau_obs = 0.3
# k=tau-tau_obs

# c=1-spatial
# f =buildF(spatial, 5, 3, 5, 3)

In [19]:
# Parameters
ndata1 = 40 #tau_obs
ndata2 = 40 #f
ndata= ndata1*ndata2 
epochs =300
ntrain = 1440
ntest = 160
batch_size = 20
gamma = 0.6
learning_rate = 0.0001
step_size= 50
# modes=12
# width=32

In [ ]:
# # Dataset generation0504
inpArr = []
outArr1 = []
outArr2 = []
tauobsArr = []
fArr = []
cArr = []

for i in range(ndata1):
        tauobsArr.append(np.random.uniform(0.1, 0.6))
for i in range(ndata2):
        f = buildF(spatial, np.random.uniform(3, 6), 3, np.random.uniform(3, 6), 3)
        fArr.append(f)
 
jj=0

for tau_obs in tauobsArr:
        for f in fArr:
                jj+=1
                if jj % 100 == 0:
                        print("Completed", jj, "/", ndata)
                
                F1, F23, Q1, Q2 = fastObserverGainCalc(f, tau_obs, 1, spatial, cof_int_x_1)
                        
                tauobs_matrix=buildtau(nx,tau_obs)
                tempin=np.concatenate([tauobs_matrix,f.reshape(1,-1)],axis=1)
                # tempout1 = np.concatenate([F1.reshape(1,-1), F23.reshape(1,-1)],axis=1)
                tempout2 = np.concatenate([Q1.reshape(1,-1), Q2.reshape(1,-1)],axis=1)
                inpArr.append(tempin)
                # outArr1.append(tempout1)
                outArr2.append(tempout2)

##########
x = np.array(inpArr)
# y1 = np.array(outArr1)
y2 = np.array(outArr2)

In [ ]:
x = x.reshape(x.shape[0],2*nx*nx)
# y1 = y1.reshape(y1.shape[0],2*nx*nx)
y2 = y2.reshape(y2.shape[0],2*nx)
np.savetxt("x_obs.dat", x)
# np.savetxt("y1_obs50_0705.dat", y1)
np.savetxt("y2_obs.dat", y2)

In [20]:
inpArr = np.loadtxt("x_obs.dat", dtype=np.float32)
outArr = np.loadtxt("y2_obs.dat", dtype=np.float32)
x = np.array(inpArr)
y = np.array(outArr)

In [21]:
grids = []
grids.append(np.linspace(0, 1, nx, dtype=np.float32))
grid=np.array(grids, dtype=np.float32).T
grid = torch.from_numpy(np.array(grid, dtype=np.float32)).cuda()

In [22]:
# Create train/test splits 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=1)
x_train = torch.from_numpy(x_train).cuda()
y_train = torch.from_numpy(y_train).cuda()
x_test = torch.from_numpy(x_test).cuda()
y_test = torch.from_numpy(y_test).cuda()

trainData = DataLoader(TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True, generator=torch.Generator(device='cuda'))
testData = DataLoader(TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False, generator=torch.Generator(device='cuda'))

In [23]:
def count_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [24]:
class BranchNet(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape
        self.conv1 = torch.nn.Conv2d(in_channels=2, out_channels=64,
                       kernel_size=5, stride=2)
        self.relu = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(in_channels=64, out_channels=128,
                       kernel_size=5, stride=2)
        self.relu = torch.nn.ReLU()
        self.fc1 = torch.nn.Linear(12800, 512)
        self.fc2 = torch.nn.Linear(512, 256)
        
    def forward(self, x):
        x = torch.reshape(x, (x.shape[0], 2, self.shape, self.shape))
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [25]:
# Define a sequential torch network for batch and trunk. Can use COV2D which we will show later
dim_x = 1
# m = int(nx*(nx+1)/2)
m = 2*nx*nx
branchobs1 = BranchNet(nx)
branchobs2 = BranchNet(nx) 
modelobs1 = dde.nn.DeepONetCartesianProd([m, branchobs1], [dim_x, 128,  256], "relu", "Glorot normal").cuda()
modelobs2 = dde.nn.DeepONetCartesianProd([m, branchobs2], [dim_x, 128,  256], "relu", "Glorot normal").cuda() 
print(count_params(modelobs1))
print(count_params(modelobs2))

6926913
6926913


In [26]:
optimizer = torch.optim.Adam([
	{'params': modelobs1.parameters(), 'lr': learning_rate,}, 
	{'params': modelobs2.parameters(), 'lr': learning_rate,}
	])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

In [27]:
# loss = nn.MSELoss()
loss = torch.nn.SmoothL1Loss(reduction='mean', beta=1.0)

train_lossArr = []
test_lossArr = []
time_Arr = []

for ep in range(epochs):
    modelobs1.train()
    modelobs2.train()
    t1 = default_timer()
    train_loss = 0
    for x, y in trainData:
        x, y = x.cuda(), y.cuda()
        y1=y[:,0:nx]
        y2=y[:,nx:2*nx]
        optimizer.zero_grad()
        out1 = modelobs1((x, grid))
        out2 = modelobs2((x, grid))
        lp = loss(out1.view(batch_size, -1), y1.view(batch_size, -1)) \
        + loss(out2.view(batch_size, -1), y2.view(batch_size, -1))  

        lp.backward()
        
        optimizer.step()
        train_loss += lp.item()
        
    scheduler.step()
    modelobs1.eval()
    modelobs2.eval()
    
    test_loss = 0
    with torch.no_grad():
        for x, y in testData:
            x, y = x.cuda(), y.cuda()
            y1=y[:,0:nx]
            y2=y[:,nx:2*nx]
            out1 = modelobs1((x, grid))
            out2 = modelobs2((x, grid))
            test_loss += loss(out1.view(batch_size, -1), y1.view(batch_size, -1)).item() \
            +loss(out2.view(batch_size, -1), y2.view(batch_size, -1)).item() 
            
    train_loss /= len(trainData)
    test_loss /= len(testData)
    
    train_lossArr.append(train_loss)
    test_lossArr.append(test_loss)
    
    t2 = default_timer()
    time_Arr.append(t2-t1)
    if ep%50 == 0:
        print(ep, t2-t1, np.mean(train_lossArr[-50:]), np.mean(test_lossArr[-50:]))

0 0.6940090860007331 0.42595003214147353 0.3055953790899366
50 0.736078798014205 0.017397229857670025 0.014980347060609347
100 0.6534281359927263 0.0008882734796134173 0.0009534758151767165
150 0.7171521919954102 0.00029999901123220404 0.00032652079456852335
200 0.7749416369770188 0.0001276632585217562 0.0001549782741466288
250 0.7952060859824996 7.167447966518618e-05 8.888264714499882e-05


In [28]:
def set_size(width, fraction=1, subplots=(1, 1), height_add=0):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float or string
            Document width in points, or string of predined document type
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy
    subplots: array-like, optional
            The number of rows and columns of subplots.
    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    if width == 'thesis':
        width_pt = 426.79135
    elif width == 'beamer':
        width_pt = 307.28987
    else:
        width_pt = width

    # Width of figure (in pts)
    fig_width_pt = width_pt * fraction
    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = height_add + fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    return (fig_width_in, fig_height_in)

tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "times",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 10,
    "font.size": 10,
    # Make the legend/bel fonts a little smaller
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
}

plt.rcParams.update(tex_fonts)

In [ ]:
# Display Model Details
plt.figure(figsize=(1000 / 300, 618 / 300), dpi=300)
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
plt.plot(train_lossArr, label="Train Loss",linewidth=1.5)
plt.plot(test_lossArr, label="Test Loss",linewidth=1.5)
plt.yscale("log")
plt.legend(fontsize="8")

testLoss = 0
trainLoss = 0
with torch.no_grad():
    for x, y in trainData:
        x, y = x.cuda(), y.cuda()
        y1=y[:,0:nx]
        y2=y[:,nx:2*nx] 
        out1 = modelobs1((x, grid))
        out2 = modelobs2((x, grid))
        out1 = out1.reshape((out1.shape[0], out1.shape[1]))
        out2 = out2.reshape((out2.shape[0], out2.shape[1]))
        trainLoss += loss(out1.view(batch_size, -1), y1.view(batch_size, -1)).item() \
            +loss(out2.view(batch_size, -1), y2.view(batch_size, -1)).item()  
        
    for x, y in testData:
        x, y = x.cuda(), y.cuda()
        y1=y[:,0:nx]
        y2=y[:,nx:2*nx] 
        out1 = modelobs1((x, grid))
        out2 = modelobs2((x, grid)) 
        out1 = out1.reshape((out1.shape[0], out1.shape[1]))
        out2 = out2.reshape((out2.shape[0], out2.shape[1])) 
        testLoss += loss(out1.view(batch_size, -1), y1.view(batch_size, -1)).item() \
            + loss(out2.view(batch_size, -1), y2.view(batch_size, -1)).item()  
    
    
print("Avg Epoch Time:", sum(time_Arr)/len(time_Arr))
print("Final Testing Loss:", testLoss)
print("Final Training Loss:", trainLoss)
plt.tick_params(labelsize=8)
# plt.savefig('img/Qloos.eps', dpi=300,bbox_inches='tight')

In [ ]:
tau = 1
tau_obs = 0.5
k=tau-tau_obs
# c=1-spatial
c=solveBetaFunction(spatial, 5, 1)
c = c-c[-1]
f =buildF(spatial, 5, 3, 5, 3)

In [ ]:
meshy, meshx = np.meshgrid(spatial, spatial)
fig = plt.figure()
ax = plt.axes(projection='3d')
surf = ax.plot_surface(meshx, meshy, zeroToNan(f))
ax.set_title(r"$f(x,y)$")

fig_intep=plt.figure()
plt.plot(spatial, c)

In [ ]:
f =buildF(spatial, 5, 3, 5, 3)

In [ ]:
F1, F23, Q1, Q2 = fastObserverGainCalc(f, tau_obs, 1, spatial, cof_int_x_1)

In [ ]:
# K,L,J=fastKernelCalc(f, c, tau, tau_obs, spatial, cof_int_x_1, cof_int_0_x)

In [ ]:
# tau_matrix=buildtau(nx,tau)
tauobs_matrix=buildtau(nx,tau_obs)

xdata=np.concatenate([tauobs_matrix,f.reshape(1,-1)],axis=1)
 
xdata = np.array(xdata, dtype=np.float32)
xdata = torch.from_numpy(xdata.reshape((1, 2*nx*nx))).cuda()

In [ ]:
Q1learning = modelobs1((xdata, grid))
Q1learning = Q1learning.detach().cpu().numpy().reshape(1, nx)
Q2learning= modelobs2((xdata, grid))
Q2learning = Q2learning.detach().cpu().numpy().reshape(1, nx)
 
####
# Q1learning=Q1learning.reshape(1,nx)
# Q2learning=Q2learning.reshape(1,nx) 

In [ ]:
fig_intep=plt.figure()
plt.plot(spatial, Q1,label=r'$Q_1(0,q)$')
plt.plot(spatial, Q1learning[0,:],label=r'$\hat Q_1$')
plt.legend(loc="upper left")
fig_intep=plt.figure()
plt.plot(spatial, Q2 ,label=r'$Q_2$')
plt.plot(spatial, Q2learning[0,:],label=r'$\hat Q_2$')
plt.legend(loc="upper left")
 


In [ ]:
path1='./Model/ObseTorchMode1.pth' 
path2='./Model/ObseTorchMode2.pth' 


In [ ]:
torch.save(modelobs1.state_dict(), path1)
torch.save(modelobs2.state_dict(), path2) 

In [ ]:
modelobs1.load_state_dict(torch.load(path1))
modelobs2.load_state_dict(torch.load(path2)) 